# Reconnaissance de Chiffres Manuscrits avec SVHN

Ce notebook utilise le dataset SVHN (Street View House Numbers) pour entraîner un modèle de reconnaissance de chiffres manuscrits. Nous allons charger les données, les prétraiter, construire un modèle de réseau de neurones convolutif, l'entraîner et l'évaluer.

## Importation des Bibliothèques

Nous commençons par importer les bibliothèques nécessaires pour notre projet.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


## Chargement et Prétraitement des Données

Nous chargeons le dataset SVHN et appliquons un prétraitement pour normaliser les images et encoder les étiquettes en one-hot.

In [2]:
# Chargement du dataset SVHN
(ds_train, ds_test), ds_info = tfds.load(
    'svhn_cropped',
    split=['train', 'test'],
    as_supervised=True,  # Retourne un tuple (img, label) au lieu d'un dictionnaire {'image': img, 'label': label}
    with_info=True  # Retourne le tuple avec les informations du dataset
)

def preprocess(images, labels):
    # Normalisation des images
    images = tf.cast(images, tf.float32) / 255.0
    # One-hot encoding des étiquettes
    labels = tf.one_hot(labels, depth=10)
    return images, labels

# Prétraitement des données
ds_train = ds_train.map(preprocess).batch(32)
ds_test = ds_test.map(preprocess).batch(32)

# Taille des images dans SVHN
hauteur, largeur, canaux = ds_info.features['image'].shape
num_classes = ds_info.features['label'].num_classes


c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...:  67%|██████▋   | 2/3 [01:15<00:37, 37.77s/ url]

## Construction du Modèle

Nous construisons un modèle de réseau de neurones convolutif pour la reconnaissance de séquences de chiffres.

In [3]:
# Construction du modèle adapté à la reconnaissance de séquences de chiffres
model = models.Sequential()

# Base convolutive pour l'extraction de caractéristiques
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(hauteur, largeur, canaux)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Adaptation pour la reconnaissance de séquences
# Ici, vous devez ajuster `sequence_length` et `features` selon la sortie de votre couche convolutive
model.add(layers.Flatten())  # Utilisez Flatten comme simplification pour cet exemple
model.add(layers.Dense(128, activation='relu'))

# Couche de sortie pour la classification de chaque chiffre dans la séquence
model.add(layers.Dense(num_classes, activation='softmax'))

# Compilation du modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


## Entraînement du Modèle

Nous configurons l'early stopping pour arrêter l'entraînement si la perte de validation ne s'améliore pas après 3 époques, puis nous entraînons le modèle.

In [4]:
# Configuration de l'Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Surveille la perte de validation
    patience=3,  # Nombre d'époques sans amélioration après lesquelles l'entraînement sera arrêté
    verbose=1,
    restore_best_weights=True  # Restaure les meilleurs poids trouvés
)

# Entraînement du modèle
historique = model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[early_stopping]  # Ajoute l'Early Stopping à la liste des callbacks
)

#model.save('./models/svhn_model.keras')

## Évaluation du Modèle

Nous évaluons les performances du modèle sur l'ensemble de test.

In [10]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(ds_test)

print(f"Perte de test: {test_loss}")
print(f"Précision de test: {test_acc}")

814/814 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8874 - loss: 0.3845
Perte de test: 0.394876092672348
Précision de test: 0.8884450197219849
